In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langgraph.graph import StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser